In [91]:
import matplotlib.pyplot as plt
import numpy as np
import math

In [223]:
# Loading CSV:

def loadCSV(filename):
    data = np.loadtxt(filename, delimiter=',', skiprows=0, unpack=False)
    # xarray = data[...,0] # First row! Need to get this on a matrix-form!
    X = data[:,range(data.shape[1]-1)] # Kolonne 1 og 2 kommer ut for 2D
    numOfColumns = data.shape[1]
    Y = data[:,data.shape[1]-1]
    ones = np.ones(X.shape[0])
    X = np.c_[ones,X] # Merge two matrices
    return data, X, Y

dataTrain, XTrain, YTrain = loadCSV('cl_train_1.csv')
dataTest, xarrayTest, YTest = loadCSV('cl_test_1.csv')


In [248]:
# Logistic Regression with Gradient descent. Defining functions:
# z = h(x)
def linearSignal(w, x):
    return (w.T).dot(x)

# Sigma, the sigmoid function. Returns a number between 0 and 1. 
def logistic(z):
    return ( 1 / (1 + math.exp(-z)))
    
# Returns a probability that the datapoint (1,x1,x2) is in y.
def prob(w,x,y):
    z = linearSignal(w,x)
    probability = (logistic(z)**y)*(1 - logistic(z))**(1-y)
    return probability

# Calculating the cross-entropy error on the training and the test-set:
def CEE(X,Y,trainedWeights):
    error = 0
    place = 0
    for y in Y:
        z = linearSignal(trainedWeights,X[place])
        sigmaZ = logistic(z)
        error = error + (y* (np.log(sigmaZ)) + (1 - y)*np.log(1-sigmaZ))
        place += 1
    return -error/((Y.shape[0]))

In [249]:
# Calculation the weights with Gradient descent:

def gradientDescent(X,Y,LR,weights,iterations):
    it = 0
    error_array = np.ones(iterations)
    placeInErrorArray = 0
    while it < iterations:
        place = 0
        summ = 0
        
        
        for x in X:
            summ = summ + np.dot((logistic(linearSignal(weights,x)) - Y[place]),x)
            place+=1
            
        # Calculating the error with CEE for each iteration:
        error = CEE(X, Y, weights)
        
        # Adding it to the error_array:
        error_array[placeInErrorArray] = error
        
        # Calc the weights for every iteration:
        weights = weights - (LR*summ)
        it+=1
        placeInErrorArray += 1
        
    return weights, error_array


In [252]:
# Using the functions to calculate values:

# Setting the initial weights in the range -0.5 to 0.5:
initWeights = np.array([0.5, 0.5,0.5])


trained_Weights, error_array = gradientDescent(XTrain,YTrain, 0.2,initWeights, 1000)

print("Trained weights: ", trained_Weights)
print("Error-array", error_array)

error1 = CEE(XTrain, YTrain, trained_Weights6)
print("Training-error: ", error1)


Trained weights:  [ 12.20510561 -34.23846311  17.66821299]
Error-array [0.75712364 1.1378614  2.34724094 0.74706378 1.55483987 0.9896409
 1.3542974  0.74866317 0.79017898 0.6133923  0.51253651 0.41614888
 0.32423411 0.27153045 0.2165233  0.18766708 0.16132461 0.14743265
 0.13690317 0.13083251 0.12648517 0.12342806 0.12096457 0.11886116
 0.1169592  0.11519257 0.11352581 0.11194125 0.11042812 0.10897911
 0.10758849 0.1062515  0.104964   0.1037224  0.10252347 0.10136438
 0.10024257 0.09915573 0.0981018  0.0970789  0.09608532 0.0951195
 0.09418001 0.09326553 0.09237487 0.09150691 0.0906606  0.089835
 0.0890292  0.08824237 0.08747374 0.08672257 0.08598817 0.0852699
 0.08456715 0.08387936 0.08320597 0.08254649 0.08190041 0.08126729
 0.08064669 0.08003818 0.07944139 0.07885592 0.07828143 0.07771757
 0.07716402 0.07662046 0.07608659 0.07556213 0.0750468  0.07454035
 0.07404252 0.07355307 0.07307176 0.07259839 0.07213272 0.07167456
 0.0712237  0.07077996 0.07034316 0.0699131  0.06948964 0.06907